# Importing dependencies

In [24]:
import pandas as pd
import os
import smart_open
from bs4 import BeautifulSoup as bs
import nltk
import re
import html2text
import gensim
from gensim import corpora
from pprint import pprint
# from gensim.utils import simple_preprocess
from smart_open import open
from gensim.models import Phrases, Word2Vec
from gensim import models
import numpy as np
from gensim.parsing.preprocessing import strip_tags
from nltk.corpus import stopwords
from gensim.models import LdaModel, LdaMulticore
# import gensim.downloader as api
from gensim.utils import simple_preprocess, lemmatize
import logging

# Using python and nltk

## Parsing html.txt files

In [25]:
with open("journal-article-10.1086_380851.html", encoding='utf8') as infile:
    html_doc = bs(infile, "html.parser")
# html_doc

## Selecting "p" tags and extracting all text then converting to lower case.

In [26]:
paragraphs=html_doc.find_all('p')
file_text = ''
for para in paragraphs:
    file_text += para.text 

# print (para.text)    
file_text = file_text.lower()
# paragraphs
# file_text

## Removing everything except letters, periods, and spaces.

In [27]:
file_text = re.sub(r'[^A-Za-z. ]', '', file_text)
# file_text

In [28]:
type(file_text)

str

# Analyzing with Gensim

## Creating a dictionary ***This can be removed.

In [29]:
dictionary = corpora.Dictionary(simple_preprocess(line, deacc=True) for line in open('journal-article-10.1086_380851.txt', encoding='utf-8'))
# strip_tags(dictionary)
print(dictionary)

2019-10-07 18:51:28,029 : INFO : built Dictionary(0 unique tokens: []) from 0 documents (total 0 corpus positions)


Dictionary(0 unique tokens: [])


In [30]:
# dictionary.token2id

## Generating bag of words corpus

The __iter__() from BoWCorpus reads a line from the file, process it to a list of words using simple_preprocess() and pass that to the dictionary.doc2bow().

The smart_open() from smart_open package because, it lets you open and read large files line-by-line


In [31]:
# nltk.download('stopwords')  # run once

class BoWCorpus(object):
    def __init__(self, path, dictionary):
        self.filepath = path
        self.dictionary = dictionary

    def __iter__(self):
        global mydict 
        for line in open(self.filepath, encoding='latin'):
            # tokenize
            tokenized_list = simple_preprocess(line, deacc=True)

            # create bag of words
            bow = self.dictionary.doc2bow(tokenized_list, allow_update=True)

            # update the source dictionary (OPTIONAL)
            mydict.merge_with(self.dictionary)

            # lazy return the BoW
            yield bow


# Create the Dictionary
mydict = corpora.Dictionary()

# Create the Corpus
bow_corpus = BoWCorpus('journal-article-10.1086_380851.txt', dictionary=mydict)  

# Print the token_id and count for each line.
for line in bow_corpus:
    print(line)

In [32]:
print(mydict)

Dictionary(0 unique tokens: [])


## Saving and loading dictionary and corpus

In [33]:
# # Save the Dict and Corpus
# mydict.save('mydict.dict') 
# corpora.MmCorpus.serialize('bow_corpus.mm', bow_corpus)  

In [34]:
# # Load them back
# loaded_dict = corpora.Dictionary.load('mydict.dict')

# corpus = corpora.MmCorpus('bow_corpus.mm')
# # for line in corpus:
# #     print(line)

## Creates a TFIDF matrix(corpus)

In [35]:
# Show the Word Weights in Corpus
for doc in bow_corpus:
    print([[mydict[id], freq] for id, freq in doc])

In [36]:
# Create the TF-IDF model
tfidf = models.TfidfModel(bow_corpus, smartirs='ntc')

2019-10-07 18:51:28,087 : INFO : collecting document frequencies
2019-10-07 18:51:28,090 : INFO : calculating IDF weights for 0 documents and 0 features (0 matrix non-zeros)


In [37]:
# Show the TF-IDF weights
for doc in tfidf[bow_corpus]:
    print([[mydict[id], np.around(freq, decimals=2)] for id, freq in doc])

## Creating bigram and trigram using Phraser models.

In [38]:
sentence_stream = [file_text.split()]
print (sentence_stream)

[['they', 'call', 'to', 'us', 'from', 'newsstands', 'and', 'racks', 'by', 'grocery', 'store', 'checkouts', 'the', 'bright', 'colors', 'on', 'their', 'covers', 'competing', 'with', 'the', 'candy', 'displays', 'teen', 'magazines', 'are', 'the', 'literary', 'sweet', 'that', 'satisfies', 'the', 'craving', 'for', 'normalcy.', 'the', 'covers', 'of', 'the', 'latest', 'teen', 'magazines', 'aimed', 'at', 'adolescent', 'girls', 'promise', 'readers', 'dates', 'beauty', 'and', 'success', 'and', 'feature', 'celebrity', 'role', 'models', 'smiling', 'beneath', 'the', 'mastheads.', 'the', 'experience', 'of', 'reading', 'the', 'glossy', 'booklets', 'seems', 'ultimately', 'depressing', 'compared', 'to', 'the', 'rich', 'superstar', 'singer', 'the', 'skinniest', 'model', 'even', 'the', 'glamorous', 'writers', 'the', 'reader', 'would', 'require', 'an', 'identity', 'facelift', 'to', 'compete', 'with', 'these', 'supposed', 'peers.', 'yet', 'teen', 'magazines', 'continue', 'to', 'flourish', 'as', 'in', 'the',

# ML with Word2Vec 

## Gensim’s word2vec expects a sequence of sentences as its input.

In [39]:
import gzip
import logging

In [40]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [41]:
sentences = [['first', 'sentence'], ['second', 'sentence']]
# train word2vec on the two sentences
model = gensim.models.Word2Vec(sentences, min_count=1)

2019-10-07 18:51:28,148 : INFO : collecting all words and their counts
2019-10-07 18:51:28,149 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-10-07 18:51:28,150 : INFO : collected 3 word types from a corpus of 4 raw words and 2 sentences
2019-10-07 18:51:28,151 : INFO : Loading a fresh vocabulary
2019-10-07 18:51:28,152 : INFO : effective_min_count=1 retains 3 unique words (100% of original 3, drops 0)
2019-10-07 18:51:28,153 : INFO : effective_min_count=1 leaves 4 word corpus (100% of original 4, drops 0)
2019-10-07 18:51:28,154 : INFO : deleting the raw counts dictionary of 3 items
2019-10-07 18:51:28,155 : INFO : sample=0.001 downsamples 3 most-common words
2019-10-07 18:51:28,156 : INFO : downsampling leaves estimated 0 word corpus (5.7% of prior 4)
2019-10-07 18:51:28,157 : INFO : estimated required memory for 3 words and 100 dimensions: 3900 bytes
2019-10-07 18:51:28,158 : INFO : resetting layer weights
2019-10-07 18:51:28,159 : INFO : training mo

## Sentence split into a list

In [42]:
from nltk.tokenize import sent_tokenize
sent_list = sent_tokenize(file_text)

## Generate a list of list of sentences

In [43]:
# list into a list of lists 
  
def extractDigits(sent_list): 
    return [[el] for el in sent_list] 
                  

list_sentlist=(extractDigits(sent_list)) 

## Generate the Word2Vec model

In [44]:
model = gensim.models.Word2Vec(list_sentlist, min_count=1)

2019-10-07 18:51:28,239 : INFO : collecting all words and their counts
2019-10-07 18:51:28,240 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-10-07 18:51:28,241 : INFO : collected 220 word types from a corpus of 220 raw words and 220 sentences
2019-10-07 18:51:28,242 : INFO : Loading a fresh vocabulary
2019-10-07 18:51:28,243 : INFO : effective_min_count=1 retains 220 unique words (100% of original 220, drops 0)
2019-10-07 18:51:28,244 : INFO : effective_min_count=1 leaves 220 word corpus (100% of original 220, drops 0)
2019-10-07 18:51:28,247 : INFO : deleting the raw counts dictionary of 220 items
2019-10-07 18:51:28,248 : INFO : sample=0.001 downsamples 220 most-common words
2019-10-07 18:51:28,248 : INFO : downsampling leaves estimated 151 word corpus (68.9% of prior 220)
2019-10-07 18:51:28,250 : INFO : estimated required memory for 220 words and 100 dimensions: 286000 bytes
2019-10-07 18:51:28,251 : INFO : resetting layer weights
2019-10-07 18:51:

In [45]:
print(model)
print(model.wv.vocab)

Word2Vec(vocab=220, size=100, alpha=0.025)
{'they call to us from newsstands and racks by grocery store checkouts the bright colors on their covers competing with the candy displays teen magazines are the literary sweet that satisfies the craving for normalcy.': <gensim.models.keyedvectors.Vocab object at 0x000001F40B3A4B70>, 'the covers of the latest teen magazines aimed at adolescent girls promise readers dates beauty and success and feature celebrity role models smiling beneath the mastheads.': <gensim.models.keyedvectors.Vocab object at 0x000001F40B782CC0>, 'the experience of reading the glossy booklets seems ultimately depressing compared to the rich superstar singer the skinniest model even the glamorous writers the reader would require an identity facelift to compete with these supposed peers.': <gensim.models.keyedvectors.Vocab object at 0x000001F40B782C50>, 'yet teen magazines continue to flourish as in the last five years traditionally womens magazines have released little si

## Set file names for train and test data

In [46]:
test_data_dir = '{}'.format(os.sep).join([gensim.__path__[0], 'test', 'test_data']) + os.sep
lee_train_file = test_data_dir + 'lee_background.cor'

In [47]:
class MyText(object):
    def __iter__(self):
        for line in open(lee_train_file):
            # assume there's one document per line, tokens separated by whitespace
            yield line.lower().split()

list_sentlist = MyText()

print(sentences)

[['first', 'sentence'], ['second', 'sentence']]


## Training the model

### min_count is for pruning the internal dictionary. Words that appear only once or twice in a large corpus are probably uninteresting typos and garbage. In addition, there’s not enough data to make any meaningful training on those words, so it’s best to ignore them.

In [48]:
model = gensim.models.Word2Vec(list_sentlist, min_count=10)

2019-10-07 18:51:28,336 : INFO : collecting all words and their counts
2019-10-07 18:51:28,340 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-10-07 18:51:28,365 : INFO : collected 10186 word types from a corpus of 59890 raw words and 300 sentences
2019-10-07 18:51:28,366 : INFO : Loading a fresh vocabulary
2019-10-07 18:51:28,371 : INFO : effective_min_count=10 retains 806 unique words (7% of original 10186, drops 9380)
2019-10-07 18:51:28,372 : INFO : effective_min_count=10 leaves 40964 word corpus (68% of original 59890, drops 18926)
2019-10-07 18:51:28,376 : INFO : deleting the raw counts dictionary of 10186 items
2019-10-07 18:51:28,377 : INFO : sample=0.001 downsamples 54 most-common words
2019-10-07 18:51:28,377 : INFO : downsampling leaves estimated 26224 word corpus (64.0% of prior 40964)
2019-10-07 18:51:28,380 : INFO : estimated required memory for 806 words and 100 dimensions: 1047800 bytes
2019-10-07 18:51:28,381 : INFO : resetting layer wei

## Saving and opening the models

In [49]:
from tempfile import mkstemp

fs, temp_path = mkstemp("gensim_temp")  # creates a temp file

model.save(temp_path)  # save the model

2019-10-07 18:51:28,623 : INFO : saving Word2Vec object under C:\Users\knolin\AppData\Local\Temp\tmpjt17ystngensim_temp, separately None
2019-10-07 18:51:28,625 : INFO : not storing attribute vectors_norm
2019-10-07 18:51:28,627 : INFO : not storing attribute cum_table
2019-10-07 18:51:28,636 : INFO : saved C:\Users\knolin\AppData\Local\Temp\tmpjt17ystngensim_temp


In [50]:
# open the model
# new_model = gensim.models.Word2Vec.load(temp_path)  

## Additional training

In [51]:
# training the model
model = gensim.models.Word2Vec(list_sentlist, size=200)

2019-10-07 18:51:28,649 : INFO : collecting all words and their counts
2019-10-07 18:51:28,651 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-10-07 18:51:28,677 : INFO : collected 10186 word types from a corpus of 59890 raw words and 300 sentences
2019-10-07 18:51:28,678 : INFO : Loading a fresh vocabulary
2019-10-07 18:51:28,686 : INFO : effective_min_count=5 retains 1723 unique words (16% of original 10186, drops 8463)
2019-10-07 18:51:28,687 : INFO : effective_min_count=5 leaves 46858 word corpus (78% of original 59890, drops 13032)
2019-10-07 18:51:28,694 : INFO : deleting the raw counts dictionary of 10186 items
2019-10-07 18:51:28,695 : INFO : sample=0.001 downsamples 49 most-common words
2019-10-07 18:51:28,696 : INFO : downsampling leaves estimated 32849 word corpus (70.1% of prior 46858)
2019-10-07 18:51:28,700 : INFO : estimated required memory for 1723 words and 200 dimensions: 3618300 bytes
2019-10-07 18:51:28,701 : INFO : resetting layer we

In [52]:
# default value of workers=3 (tutorial says 1...)
model = gensim.models.Word2Vec(list_sentlist, workers=4)

2019-10-07 18:51:29,050 : INFO : collecting all words and their counts
2019-10-07 18:51:29,051 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-10-07 18:51:29,076 : INFO : collected 10186 word types from a corpus of 59890 raw words and 300 sentences
2019-10-07 18:51:29,077 : INFO : Loading a fresh vocabulary
2019-10-07 18:51:29,087 : INFO : effective_min_count=5 retains 1723 unique words (16% of original 10186, drops 8463)
2019-10-07 18:51:29,087 : INFO : effective_min_count=5 leaves 46858 word corpus (78% of original 59890, drops 13032)
2019-10-07 18:51:29,099 : INFO : deleting the raw counts dictionary of 10186 items
2019-10-07 18:51:29,100 : INFO : sample=0.001 downsamples 49 most-common words
2019-10-07 18:51:29,101 : INFO : downsampling leaves estimated 32849 word corpus (70.1% of prior 46858)
2019-10-07 18:51:29,106 : INFO : estimated required memory for 1723 words and 100 dimensions: 2239900 bytes
2019-10-07 18:51:29,107 : INFO : resetting layer we

## Using the model
Word2Vec supports several word similarity tasks out of the box:

In [61]:
model.wv.most_similar(positive=['read'], negative=['write'], topn=1)

KeyError: "word 'write' not in vocabulary"

In [54]:
print(model.wv.similarity('read', 'party'))
print(model.wv.similarity('seen', 'murder'))

0.9922514
0.9976951


In [55]:
print(model.predict_output_word(['read', 'seen', 'received']))

[('first', 0.0006879926), ('government', 0.0006853297), ('us', 0.00067960547), ('also', 0.00067762344), ('when', 0.0006769003), ('arafat', 0.00067580084), ('wales', 0.00067426875), ('there', 0.0006738335), ('all', 0.0006729461), ('if', 0.00067110173)]


In [56]:
model['read']  # raw NumPy vector of a word

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([ 0.01279023,  0.03632816,  0.03516942,  0.00794312,  0.00783985,
        0.01547673,  0.04271121,  0.02785933, -0.00074814, -0.07772044,
       -0.01446604, -0.01959375, -0.04014302,  0.02537696, -0.00258341,
       -0.00754798,  0.03231826, -0.02489292, -0.01045883, -0.0131178 ,
        0.02734876, -0.00216641,  0.02051614,  0.01951203, -0.023349  ,
        0.00093485, -0.00204796,  0.03475473,  0.02016631, -0.00863217,
       -0.00229754, -0.02820375,  0.00020949, -0.00806989,  0.00634657,
       -0.03431857,  0.03278439,  0.0082471 , -0.01915531, -0.0187015 ,
        0.02205434, -0.01223822, -0.03114535,  0.03396862,  0.01340313,
        0.00366304, -0.0053626 ,  0.00701275, -0.00726209, -0.00190935,
       -0.03907422,  0.01314985,  0.00981733, -0.02849872, -0.01973668,
        0.0120859 ,  0.01522844, -0.03015096,  0.02332699,  0.02463402,
        0.00495318, -0.00889076,  0.0001364 , -0.02757069, -0.02030646,
        0.04398051, -0.0165002 ,  0.01805867, -0.01477974,  0.04